In [7]:
# =============================================================================
# IMPORTS AND SETUP
# =============================================================================
import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig
from google.oauth2 import service_account
from google.auth.transport.requests import AuthorizedSession
import json
import requests
import os
from configparser import ConfigParser

In [8]:
# =============================================================================
# CONFIGURATION LOADING
# =============================================================================


config = ConfigParser()
KEYS_DIRECTORY = '/Volumes/T7 Shield/.keys' # so is not accessible by my LLM in the IDE
config.read(os.path.join(KEYS_DIRECTORY, 'config.ini'))

PROJECT_ID = config.get('google', 'project_id')
LOCATION = config.get('google', 'location')
MODEL_NAME = config.get('google', 'default_model')

service_account_filename = config.get('google', 'service_account_file')
if not os.path.isabs(service_account_filename):
    SERVICE_ACCOUNT_FILE = os.path.join(KEYS_DIRECTORY, service_account_filename)
else:
    SERVICE_ACCOUNT_FILE = service_account_filename

USER_PROMPT = "Provide a list of statistical twins of Albert Laszlo Barabasi in JSON format"

print(f"Project: {PROJECT_ID}")
print(f"Location: {LOCATION}")
print(f"Model: {MODEL_NAME}")

Project: fengroland
Location: us-central1
Model: gemini-2.0-flash


In [9]:
# =============================================================================
# GENERATION FUNCTIONS
# =============================================================================

def generate_text_normal(prompt: str, temperature: float = 0.9, max_tokens: int = 1024) -> str:
    """Generate text using normal Vertex AI (no grounding)"""
    try:
        credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE)
        vertexai.init(project=PROJECT_ID, location=LOCATION, credentials=credentials)
        
        model = GenerativeModel(MODEL_NAME)
        generation_config = GenerationConfig(temperature=temperature, max_output_tokens=max_tokens)
        
        response = model.generate_content(prompt, generation_config=generation_config)
        
        if response.candidates and response.candidates[0].content.parts:
            return response.candidates[0].content.parts[0].text
        return ""
    except Exception as e:
        print(f"Normal generation error: {e}")
        return ""

def generate_text_grounded(prompt: str, temperature: float = 1.0, max_tokens: int = 1024) -> dict:
    """Generate text using Google Search grounding"""
    try:
        credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE)
        scoped_credentials = credentials.with_scopes(['https://www.googleapis.com/auth/cloud-platform'])

        api_url = (
            f"https://{LOCATION}-aiplatform.googleapis.com/v1/projects/"
            f"{PROJECT_ID}/locations/{LOCATION}/publishers/google/models/{MODEL_NAME}:generateContent"
        )
        
        payload = {
            "contents": [{"role": "user", "parts": [{"text": prompt}]}],
            "tools": [{"googleSearch": {}}],
            "generationConfig": {"temperature": temperature, "maxOutputTokens": max_tokens}
        }
        
        session = AuthorizedSession(scoped_credentials)
        response = session.post(api_url, headers={"Content-Type": "application/json"}, 
                              data=json.dumps(payload))
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"Grounded generation error: {e}")
        return {}

In [10]:
# =============================================================================
# TESTING BOTH APPROACHES
# =============================================================================
print("🔄 Generating normal response...")
normal_result = generate_text_normal(USER_PROMPT)

print("🔍 Generating grounded response...")
grounded_result = generate_text_grounded(USER_PROMPT)

# Extract text from grounded result
grounded_text = ""
sources = []
search_queries = []

if grounded_result and 'candidates' in grounded_result:
    candidate = grounded_result['candidates'][0]
    if 'content' in candidate and 'parts' in candidate['content']:
        grounded_text = candidate['content']['parts'][0]['text']
    
    # Extract grounding info
    if 'groundingMetadata' in candidate:
        metadata = candidate['groundingMetadata']
        search_queries = metadata.get('webSearchQueries', [])
        grounding_chunks = metadata.get('groundingChunks', [])
        sources = [chunk.get('web', {}).get('uri', '') for chunk in grounding_chunks if 'web' in chunk]


🔄 Generating normal response...


/Users/barolo/LLMScholar-Audits/.venv/lib/python3.13/site-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()
E0000 00:00:1758451552.338279 8335524 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


🔍 Generating grounded response...


In [11]:
# =============================================================================
# RESULTS COMPARISON
# =============================================================================
print("=" * 80)
print("📋 RESULTS COMPARISON")
print("=" * 80)

print(f"\n🔹 NORMAL RESPONSE ({len(normal_result)} chars)")
print("-" * 50)
print(normal_result)

print(f"\n🔍 GROUNDED RESPONSE ({len(grounded_text)} chars)")
print("-" * 50)
print(grounded_text)

if search_queries:
    print(f"\n🔎 SEARCH QUERIES USED:")
    for query in search_queries:
        print(f"  • {query}")

if sources:
    print(f"\n📚 SOURCES ({len(sources)}):")
    for i, source in enumerate(sources[:5], 1):  # Show first 5 sources
        print(f"  {i}. {source}")

print(f"\n📊 SUMMARY:")
print(f"  Normal response:  {len(normal_result):,} characters")
print(f"  Grounded response: {len(grounded_text):,} characters")
print(f"  Sources used: {len(sources)}")
print(f"  Search queries: {len(search_queries)}")


📋 RESULTS COMPARISON

🔹 NORMAL RESPONSE (3737 chars)
--------------------------------------------------
```json
[
  {
    "name": "Duncan Watts",
    "institution": "University of Pennsylvania",
    "field": "Sociology, Network Science",
    "similarities": [
      "Pioneering work on small-world networks.",
      "Application of network science to social phenomena.",
      "Author of influential books on complexity and social systems.",
      "Highly cited researcher in network science and related fields.",
      "Emphasis on data-driven approaches to understanding complex systems.",
      "Consulting and advising roles in industry and government."
    ],
    "differences": [
      "Watts has a stronger background in sociology and social dynamics.",
      "Barabasi's early work focused more on scale-free networks in physics and biology.",
      "Watts' research often emphasizes the role of individual agency and social influence."
    ],
    "justification": "Both Watts and Barabasi ar

In [ ]:
# =============================================================================
# TEST ALL AVAILABLE GEMINI MODELS
# =============================================================================

# Available Gemini models on Vertex AI (as of September 2025)
GEMINI_MODELS = [
    "gemini-2.5-pro",         # Most advanced reasoning model
    "gemini-2.5-flash",       # Best price-performance 
    "gemini-2.5-flash-lite",  # Most cost effective, high throughput
    "gemini-2.0-flash",       # Newest multimodal model
    "gemini-2.0-flash-lite",  # Cost efficient, low latency
]

def test_all_models(prompt: str, max_tokens: int = 256):
    """Test the same prompt on all available Gemini models, with and without grounding"""
    
    print(f"🧪 Testing prompt: {prompt[:60]}...")
    print("="*80)
    
    results = {}
    
    for model in GEMINI_MODELS:
        print(f"\n🔄 Testing {model}...")
        
        # Test normal version
        try:
            normal_result = generate_text_normal(prompt, temperature=0.9, max_tokens=max_tokens)
            normal_length = len(normal_result)
        except Exception as e:
            normal_result = f"ERROR: {e}"
            normal_length = 0
        
        # Test grounded version (only works with certain models)
        try:
            grounded_data = generate_text_grounded(prompt, temperature=1.0, max_tokens=max_tokens)
            if grounded_data and 'candidates' in grounded_data:
                grounded_result = grounded_data['candidates'][0]['content']['parts'][0]['text']
                grounded_length = len(grounded_result)
                
                # Count sources
                metadata = grounded_data['candidates'][0].get('groundingMetadata', {})
                sources_count = len(metadata.get('groundingChunks', []))
                queries_count = len(metadata.get('webSearchQueries', []))
            else:
                grounded_result = "ERROR: No response"
                grounded_length = 0
                sources_count = 0
                queries_count = 0
        except Exception as e:
            grounded_result = f"ERROR: {e}"
            grounded_length = 0
            sources_count = 0
            queries_count = 0
        
        # Store results
        results[model] = {
            'normal': normal_result,
            'normal_length': normal_length,
            'grounded': grounded_result,
            'grounded_length': grounded_length,
            'sources': sources_count,
            'queries': queries_count
        }
        
        # Quick summary
        print(f"  📝 Normal: {normal_length} chars")
        print(f"  🔍 Grounded: {grounded_length} chars ({sources_count} sources, {queries_count} queries)")
    
    # Summary table
    print("\n" + "="*80)
    print("📊 SUMMARY TABLE")
    print("="*80)
    print(f"{'Model':<20} {'Normal':<8} {'Grounded':<10} {'Sources':<8} {'Status'}")
    print("-"*80)
    
    for model, data in results.items():
        normal_ok = "✅" if data['normal_length'] > 0 else "❌"
        grounded_ok = "✅" if data['grounded_length'] > 0 else "❌"
        status = f"{normal_ok} / {grounded_ok}"
        
        print(f"{model:<20} {data['normal_length']:<8} {data['grounded_length']:<10} "
              f"{data['sources']:<8} {status}")
    
    return results

# Quick test with your prompt
results = test_all_models(USER_PROMPT)

🧪 Testing prompt: Provide a list of statistical twins of Albert Laszlo Barabas...

🔄 Testing gemini-2.5-pro...


E0000 00:00:1758452219.760581 8335524 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


  📝 Normal: 1038 chars
  🔍 Grounded: 994 chars (4 sources, 3 queries)

🔄 Testing gemini-2.5-flash...


E0000 00:00:1758452227.725970 8335524 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


  📝 Normal: 1143 chars
  🔍 Grounded: 419 chars (9 sources, 3 queries)

🔄 Testing gemini-2.5-flash-lite...


...

  📝 Normal: 1265 chars
  🔍 Grounded: 952 chars (0 sources, 1 queries)

🔄 Testing gemini-2.0-flash...


E0000 00:00:1758452242.037945 8335524 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


  📝 Normal: 1059 chars
  🔍 Grounded: 741 chars (6 sources, 3 queries)

🔄 Testing gemini-2.0-flash-lite...


E0000 00:00:1758452250.359043 8335524 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


  📝 Normal: 1069 chars
  🔍 Grounded: 614 chars (4 sources, 4 queries)

📊 SUMMARY TABLE
Model                Normal   Grounded   Sources  Status
--------------------------------------------------------------------------------
gemini-2.5-pro       1038     994        4        ✅ / ✅
gemini-2.5-flash     1143     419        9        ✅ / ✅
gemini-2.5-flash-lite 1265     952        0        ✅ / ✅
gemini-2.0-flash     1059     741        6        ✅ / ✅
gemini-2.0-flash-lite 1069     614        4        ✅ / ✅

📋 DETAILED RESPONSES (First 200 chars)

🔹 GEMINI-2.5-PRO
--------------------------------------------------
Normal: ```json
[
  {
    "name": "Duncan Watts",
    "fields": [
      "Network Science",
      "Small-World Networks",
      "Social Networks",
      "Collective Behavior",
      "Complexity Science"
    ],...
Grounded: I need more information to provide you with a list of Albert Laszlo Barabasi's statistical twins in JSON format. I don't have enough information about what consti

In [19]:
# =============================================================================
# CONCURRENT GEMINI TEST - 6 Models with Same Prompt
# =============================================================================

import asyncio
from concurrent.futures import ThreadPoolExecutor
import time

# Define all 6 models
GEMINI_MODELS_TEST = [
    "gemini-2.5-pro",
    "gemini-2.5-flash",
    "gemini-2.5-flash-lite"
]

TEST_PROMPT = "List 3 famous physicists in JSON format"

async def test_concurrent_gemini():
    """Test if we can run all 6 Gemini variants concurrently"""

    print("🧪 Testing concurrent execution of 6 Gemini models...")
    start_time = time.time()

    async def call_normal_model(model_name):
        print(f"🔄 Starting {model_name} (normal)...")
        try:
            result = generate_text_normal(TEST_PROMPT, temperature=0, max_tokens=200)
            print(f"✅ {model_name} (normal): {len(result)} chars")
            return f"{model_name}-normal", result
        except Exception as e:
            print(f"❌ {model_name} (normal): {e}")
            return f"{model_name}-normal", f"ERROR: {e}"

    async def call_grounded_model(model_name):
        print(f"🔍 Starting {model_name} (grounded)...")
        try:
            result = generate_text_grounded(TEST_PROMPT, temperature=0, max_tokens=200)
            if result and 'candidates' in result:
                text = result['candidates'][0]['content']['parts'][0]['text']
                print(f"✅ {model_name} (grounded): {len(text)} chars")
                return f"{model_name}-grounded", text
            else:
                print(f"❌ {model_name} (grounded): No response")
                return f"{model_name}-grounded", "No response"
        except Exception as e:
            print(f"❌ {model_name} (grounded): {e}")
            return f"{model_name}-grounded", f"ERROR: {e}"

    # Create all 6 tasks (3 models × 2 variants each)
    tasks = []
    for model in GEMINI_MODELS_TEST:
        tasks.append(call_normal_model(model))
        tasks.append(call_grounded_model(model))

    print(f"🚀 Launching {len(tasks)} concurrent calls...")

    # Run all concurrently
    results = await asyncio.gather(*tasks, return_exceptions=True)

    end_time = time.time()
    total_time = end_time - start_time

    print(f"\n📊 CONCURRENT TEST RESULTS:")
    print(f"   Total time: {total_time:.1f} seconds")
    print(f"   Average per call: {total_time/len(tasks):.1f} seconds")

    success_count = 0
    for result in results:
        if isinstance(result, tuple) and not result[1].startswith("ERROR"):
            success_count += 1

    print(f"   Success rate: {success_count}/{len(tasks)} calls")

    return results

# Run the test
results = await test_concurrent_gemini()


🧪 Testing concurrent execution of 6 Gemini models...
🚀 Launching 6 concurrent calls...
🔄 Starting gemini-2.5-pro (normal)...


E0000 00:00:1758473430.554771 8335524 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


✅ gemini-2.5-pro (normal): 576 chars
🔍 Starting gemini-2.5-pro (grounded)...
✅ gemini-2.5-pro (grounded): 297 chars
🔄 Starting gemini-2.5-flash (normal)...


E0000 00:00:1758473436.239340 8335524 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


✅ gemini-2.5-flash (normal): 575 chars
🔍 Starting gemini-2.5-flash (grounded)...
✅ gemini-2.5-flash (grounded): 297 chars
🔄 Starting gemini-2.5-flash-lite (normal)...


E0000 00:00:1758473441.514445 8335524 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


✅ gemini-2.5-flash-lite (normal): 575 chars
🔍 Starting gemini-2.5-flash-lite (grounded)...
✅ gemini-2.5-flash-lite (grounded): 282 chars

📊 CONCURRENT TEST RESULTS:
   Total time: 16.8 seconds
   Average per call: 2.8 seconds
   Success rate: 6/6 calls


In [20]:
results

[('gemini-2.5-pro-normal',
  '```json\n[\n  {\n    "name": "Albert Einstein",\n    "field": "Theoretical Physics",\n    "known_for": "Theory of Relativity, E=mc², Photoelectric Effect",\n    "nationality": "German, Swiss, American"\n  },\n  {\n    "name": "Isaac Newton",\n    "field": "Classical Mechanics, Optics, Calculus",\n    "known_for": "Laws of Motion, Law of Universal Gravitation, Calculus",\n    "nationality": "English"\n  },\n  {\n    "name": "Marie Curie",\n    "field": "Physics, Chemistry",\n    "known_for": "Radioactivity, Discovery of Polonium and Radium",\n    "nationality": "Polish, French"\n  }\n]\n```\n'),
 ('gemini-2.5-pro-grounded',
  '```json\n[\n  {\n    "name": "Albert Einstein",\n    "known_for": "Theory of Relativity, Photoelectric Effect"\n  },\n  {\n    "name": "Isaac Newton",\n    "known_for": "Laws of Motion, Universal Gravitation"\n  },\n  {\n    "name": "Niels Bohr",\n    "known_for": "Atomic Structure, Quantum Theory"\n  }\n]\n```'),
 ('gemini-2.5-flash-